<a href="https://colab.research.google.com/github/Annagmo/treinamento-h2ia/blob/main/Relat%C3%B3rio_Buscas_sem_informa%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O Problema
Sliding Puzzle - Bloco Deslizante

In [58]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

## Busca em largura

In [119]:
import numpy as np
from random import sample

tabuleiro = np.array([[1,2,3]])

matriz = sample(range(4, 7), 3)
tabuleiro_aleat = np.array([matriz[i:i + 3] for i in range(0, len(matriz), 3)]) #inicializa uma matriz randômica de 0-3 e o método sample garante que é sem números repetidos
tabuleiro = np.vstack((tabuleiro,tabuleiro_aleat)) 
tabuleiro = np.vstack((tabuleiro,[7,8,0]))
tabuleiro #assim conseguimos que os 3 últimos nodos sejam randômicos mas não o tabuleiro inteiro, o que o colaboratory poderia não suportar.

array([[1, 2, 3],
       [6, 4, 5],
       [7, 8, 0]])

In [120]:
#Para checar se o tabuleiro randomizado é solucionável:
#A seguir está uma regra simples para verificar se um quebra-cabeça de 8 é solucionável.
#Checamos as inversões. Não é possível resolver uma instância de 8 quebra-cabeças se o número de inversões for ímpar no estado de entrada.
def getInvCount(arr):
    contador_de_inv = 0
    empty_value = -1
    for i in range(0, 9):
        for j in range(i + 1, 9):
            if arr[j] != empty_value and arr[i] != empty_value and arr[i] > arr[j]:
                contador_de_inv += 1
    return contador_de_inv
 
     

def isSolvable(tabuleiro) : #retorna true ou false
 
    # conta as inversões em um jogo de 8 quadrados
    contador_de_inv = getInvCount([j for sub in tabuleiro for j in sub])
 
    # verdadeiro se contador de inversão é par
    return (contador_de_inv % 2 == 0)
     
print(isSolvable(tabuleiro))

     

True


In [121]:
class Estado_do_Jogo:
  def __init__(self, state):
    self.state = state
    self.where_is_zero()

  def check(self):
    compare = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 0]])
    comparison = self.state == compare
    return comparison.all()  
  
  def check_moves(self):
    possibles_moves = []
    if( self.column > 0 ):
      possibles_moves.append('a')
    if( self.column < 2 ):
      possibles_moves.append('d')
    if( self.row > 0 ):
      possibles_moves.append('w')
    if( self.row < 2 ):
      possibles_moves.append('s')
    return possibles_moves

  def where_is_zero(self):
    self.row, self.column = np.where( self.state == 0)


  def move(self, position):
    self.where_is_zero()
    move_done = self.state.copy()

    if(position == 'w'):
      value = move_done[self.row-1, self.column]
      move_done[self.row, self.column] = value
      move_done[self.row-1, self.column] = 0
      
      return move_done
    if(position == 'd'):
      value = move_done[self.row, self.column+1]
      move_done[self.row, self.column] = value
      move_done[self.row, self.column+1] = 0
      self.column += 1
      
      return move_done

    if(position == 's'):
      value = move_done[self.row+1, self.column]
      move_done[self.row, self.column] = value
      move_done[self.row+1, self.column] = 0
      self.row += 1
      
      return move_done

    if(position == 'a'):
      value = move_done[self.row, self.column-1]
      move_done[self.row, self.column] = value
      move_done[self.row, self.column-1] = 0
      self.column -= 1
      return move_done

    return move_done

In [122]:
class Node:
  def __init__(self, state):
    self.state = Estado_do_Jogo(state)
    self.children = []
  
  def parent_node(self, node):
    self.parent = node
    
  def children_node(self, child):
    self.children.append(child)

In [123]:
def visited_node(locked_nodes, node):
    for actual_node in locked_nodes:
      comparison = actual_node == node.state.state
      if comparison.all():
        return True
    return False

In [124]:
def put_children(node, locked_nodes, list):
  node.children = []
  moves = node.state.check_moves()
  for move in moves:
    child = node.state.move(move)
    child_node = Node(child)
    if not visited_node(locked_nodes, child_node):
      node.children_node(child_node)
      child_node.parent_node(node)
      list.append(child_node)
  return node

In [125]:
# busca em largura
stack = []
queue = []
locked_nodes = []
movements = 0
node = Node(tabuleiro)

node = put_children(node, locked_nodes, queue)
movements += 1

node = queue.pop(0)
if(isSolvable(tabuleiro)):
  while (len(queue) != 0):
    if (node.state.check()):
      print(movements)
      break
    else:
      locked_nodes.append(node.state.state.copy())
      node = put_children(node, locked_nodes, queue)
      movements += 1
      node = queue.pop(0)
else:
  print('O tabuleiro não tem solução.')

4624


## Busca em Profundidade

## Discorra sobre o desempenho dos métodos em questões de:


1.   Consumo de memória
2.   Processamento

